<a href="https://colab.research.google.com/github/JonatanSiracusa/download-historical-series/blob/main/download_hist_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Historical Prices

# 1. Kick-off: Libraries Importing, Variables Setup and Functions

### 1.1 Libraries and Variables setup

In [1]:
# import numpy as np
# import pandas as pd
# import copy
# import scipy
# from scipy import stats
# from scipy.stats import gaussian_kde
# from fitter import Fitter
# import math
# import random
# import time
# import datetime as dt
# from datetime import datetime

# import yfinance as yf

# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# from plotly.offline import init_notebook_mode
# init_notebook_mode(connected=True)

# import cufflinks as cf
# cf.set_config_file(offline=True, dimensions=(800,500))

# from IPython.display import display, HTML, Javascript
# from src.bloomberg_template import set_bloomberg_dark, set_bloomberg_light, get_color, get_colorscale

# import warnings
# warnings.simplefilter("ignore")

In [2]:
# import pandas as pd
# import numpy as np
# import time
# import datetime as dt
# from datetime import datetime

# import yfinance as yf

# import warnings
# warnings.filterwarnings('ignore')

# import cufflinks as cf
# cf.set_config_file(offline=True)
# #cf.go_offline()

# # lo usamos en la visualizacion custom
# from IPython.display import display, HTML, Javascript

# # medimos el tiempo de ejecucion del programa
# star_time = time.time()

In [3]:
# import pandas as pd
# import cufflinks as cf
# # from plotly.subplots import make_subplots

# # Activar offline mode de cufflinks
# cf.go_offline()




# # cf.set_config_file(offline=True, dimensions=(800,500))

# from IPython.display import display, HTML, Javascript

# from src.bloomberg_template import set_bloomberg_dark, set_bloomberg_light, get_color, get_colorscale
# from src.utils import descargar_datos_yf
# from src.ticker_data import TickerData as td

In [4]:
import pandas as pd
import numpy as np
import time
import datetime as dt
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf

from IPython.display import display, HTML, Javascript

import yfinance as yf

import warnings
warnings.filterwarnings('ignore')

from src.bloomberg_template import set_bloomberg_dark, set_bloomberg_light, get_color, get_colorscale
from src.utils import descargar_datos_yf
from src.ticker_data import TickerData as td
from src.quant_fi import calcular_ema


Setting the parameters:

In [5]:
# Medimos el tiempo de ejecucion del programa
star_time = time.time()

MARKET_DAYS_YEAR = 252
EXPORT_DATA = False

TICKERS = ['GGAL']
TICKERS_YF = ['GGAL.BA']

START_DATE = '2023-12-11'
END_DATE = ''

### 1.2 Functions

# 2. Data Loading

In [6]:
tickers = TICKERS
tickers_yf = TICKERS_YF

start_date = datetime.strptime(START_DATE, '%Y-%m-%d')
end_date = datetime.now() if not END_DATE else datetime.strptime(END_DATE, "%Y-%m-%d")

raw_data = descargar_datos_yf(tickers_yf, start_date, end_date)
prices = raw_data.copy()

prices.index = prices.index.strftime('%Y-%m-%d')
prices.index = pd.to_datetime(prices.index)

print('Index type:', prices.index.dtype)
prices.head()


1 Failed download:
['GGAL.BA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Sin datos: GGAL.BA


AttributeError: 'RangeIndex' object has no attribute 'strftime'

In [ ]:
datos_2 = pd.DataFrame({
    'open': [1685.0, 1787.0, 1830.0, 1835.0, 1790.0, 7860.0, 7520.0, 7610.0, 7420.0, 7170.0],
    'high': [1783.75, 1833.75, 1921.0, 1903.0, 1799.949951, 7930.0, 7730.0, 7690.0, 7470.0, 7340.0],
    'low': [1660.0, 1733.5, 1705.0, 1780.0, 1640.0, 7540.0, 7520.0, 7310.0, 7240.0, 7070.0],
    'close': [1756.349976, 1780.849976, 1827.300049, 1788.849976, 1652.5, 7580.0, 7610.0, 7430.0, 7290.0, 7190.0],
}, index=pd.to_datetime([
    '2023-12-11', '2023-12-12', '2023-12-13', '2023-12-14', '2023-12-15',
    '2025-04-24', '2025-04-25', '2025-04-28', '2025-04-29', '2025-04-30'
]))

datos_2

,open,high,low,close
2023-12-11,1685.0,1783.750000,1660.0,1756.349976
2023-12-12,1787.0,1833.750000,1733.5,1780.849976
2023-12-13,1830.0,1921.000000,1705.0,1827.300049
2023-12-14,1835.0,1903.000000,1780.0,1788.849976
2023-12-15,1790.0,1799.949951,1640.0,1652.500000
2025-04-24,7860.0,7930.000000,7540.0,7580.000000
2025-04-25,7520.0,7730.000000,7520.0,7610.000000
2025-04-28,7610.0,7690.000000,7310.0,7430.000000
2025-04-29,7420.0,7470.000000,7240.0,7290.000000
2025-04-30,7170.0,7340.000000,7070.0,7190.000000


# 3. Data Cleaning

En esta etapa realizaremos lo siguiente:

1.	Modificamos los nombres de los Tickers, para mayor comodidad.

2.	Creamos DF limpios de todos los activos con precios OHLCAV.

3.	Hacemos un Checkpoint y guardamos lo realizado hasta el momento.

Cambiamos los nombres de los activos.

In [ ]:
for ticker_y, name in zip(tickers_yf, tickers):
	prices.rename(columns={ticker_y: name}, inplace=True)

df = prices
print("\nNiveles del índice en columnas:")
for i, name in enumerate(df.columns.names):
	print(f"Nivel {i} ({name}): {df.columns.unique(level=i).tolist()}")

prices.head()


Niveles del índice en columnas:
Nivel 0 (None): ['GGAL']
Nivel 1 (Price): ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
Nivel 2 (Ticker): ['GGAL']


GGAL                                                   
Price         Adj Close        Close         High     Low    Open   Volume
Ticker             GGAL         GGAL         GGAL    GGAL    GGAL     GGAL
Date                                                                      
2023-12-11  1706.043701  1756.349976  1783.750000  1660.0  1685.0  4215047
2023-12-12  1729.841919  1780.849976  1833.750000  1733.5  1787.0  1965001
2023-12-13  1774.961548  1827.300049  1921.000000  1705.0  1830.0  3411926
2023-12-14  1737.612793  1788.849976  1903.000000  1780.0  1835.0  3493588
2023-12-15  1605.168213  1652.500000  1799.949951  1640.0  1790.0  3948715

In [ ]:
tipo_precio = df.columns.unique(level=1).tolist()
tipo_precio

['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

In [ ]:
data = td(tickers, prices, tipo_precio)
data.load_ohlcav()

Se agregaron a la Class los DataFrame de GGAL.
Se insertaron las columnas OHLCAV en los DF GGAL.


# 4. Data Transformation

### Dataframe de cada activo

Calculamos y agregamos a cada DF de los activos, lo siguiente:

*	Retornos logaritmicos
*	Volatilidad de las ultimas 40 ruedas, anualizada.

In [ ]:
from src.utils import insert_ohlc_cols, insert_ohlcav_cols, calc_returns_df, calc_returns_volat_df, calculate_returns, calc_prices_df, calculate_volat

In [ ]:
# Agregamos los retornos logaritmicos
data.add_columns(calculate_returns, method='log', column_price='Close')

# Agregamos la volatilidad anual
data.add_columns(calculate_volat, method='log', column_price='Close', window=40, market_days_year=MARKET_DAYS_YEAR)

Agregados a los DF de la Class usando "calculate_returns": Log Returns sobre Close en "returns".
Agregados a los DF de la Class usando "calculate_volat": Volatilidad Anual con ventana de 40 ruedas en función de Log Returns sobre Close en "volat_40".


In [ ]:
data.GGAL

,Open,High,Low,Close,Adj Close,Volume,log_returns,volat_40
Date,,,,,,,,
2023-12-11,1685.0,1783.750000,1660.0,1756.349976,1706.043701,4215047,0.000000,0.000000
2023-12-12,1787.0,1833.750000,1733.5,1780.849976,1729.841919,1965001,0.013853,0.000000
2023-12-13,1830.0,1921.000000,1705.0,1827.300049,1774.961548,3411926,0.025749,0.000000
2023-12-14,1835.0,1903.000000,1780.0,1788.849976,1737.612793,3493588,-0.021267,0.000000
2023-12-15,1790.0,1799.949951,1640.0,1652.500000,1605.168213,3948715,-0.079284,0.000000
...,...,...,...,...,...,...,...,...
2025-04-25,7520.0,7730.000000,7520.0,7610.000000,7610.000000,2022374,0.003950,0.666477
2025-04-28,7610.0,7690.000000,7310.0,7430.000000,7430.000000,1468569,-0.023937,0.667739
2025-04-29,7420.0,7470.000000,7240.0,7290.000000,7290.000000,1631781,-0.019022,0.668941


In [ ]:
# Checkpoint
data.backup_dataframes(suffix='_v1')
data.backup_dataframes_to_globals(suffix='_v1')

Se crearon copias INTERNAS de la INSTANCIA de los DF:
GGAL_v1.

Se crearon copias GLOBALES de los DF:
GGAL_v1.



# 5. Visualization

In [ ]:
datos = data.GGAL[['Open', 'High', 'Low', 'Close', 'Volume']]
datos

,Open,High,Low,Close,Volume
Date,,,,,
2023-12-11,1685.0,1783.750000,1660.0,1756.349976,4215047
2023-12-12,1787.0,1833.750000,1733.5,1780.849976,1965001
2023-12-13,1830.0,1921.000000,1705.0,1827.300049,3411926
2023-12-14,1835.0,1903.000000,1780.0,1788.849976,3493588
2023-12-15,1790.0,1799.949951,1640.0,1652.500000,3948715
...,...,...,...,...,...
2025-04-25,7520.0,7730.000000,7520.0,7610.000000,2022374
2025-04-28,7610.0,7690.000000,7310.0,7430.000000,1468569
2025-04-29,7420.0,7470.000000,7240.0,7290.000000,1631781


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(
    x=datos.index,
    open=datos['Open'],
    high=datos['High'],
    low=datos['Low'],
    close=datos['Close'],
    increasing_line_color='blue',  # Velas alcistas
    decreasing_line_color='red'    # Velas bajistas
)])

fig.update_layout(
    title='Gráfico de Velas',
    xaxis_title='Fecha',
    yaxis_title='Precio',
    width=1000,
    height=600,
    font=dict(family='calibri', size=14),
    xaxis_rangeslider_visible=False  # Oculta el rango inferior si querés
)

fig.show()

1

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import ta

# === Simulación de datos: reemplazalo por tu DataFrame real
# datos = pd.read_csv('archivo.csv', index_col='Date', parse_dates=True)

# Asegurar que el índice sea fecha
datos.index = pd.to_datetime(datos.index)

# Filtrar solo días hábiles
datos = datos[datos.index.dayofweek < 5]

# === Calcular indicadores técnicos ===
# EMA
datos['EMA_9'] = datos['Close'].ewm(span=9, adjust=False).mean()
datos['EMA_21'] = datos['Close'].ewm(span=21, adjust=False).mean()

# Bandas de Bollinger
bb = ta.volatility.BollingerBands(close=datos['Close'], window=20, window_dev=2)
datos['BB_upper'] = bb.bollinger_hband()
datos['BB_middle'] = bb.bollinger_mavg()
datos['BB_lower'] = bb.bollinger_lband()

# RSI
datos['RSI'] = ta.momentum.RSIIndicator(close=datos['Close'], window=14).rsi()

# === Crear el gráfico de velas ===
fig = go.Figure()

# Velas japonesas
fig.add_trace(go.Candlestick(
    x=datos.index,
    open=datos['Open'],
    high=datos['High'],
    low=datos['Low'],
    close=datos['Close'],
    name='Velas'
))

# EMA 9
fig.add_trace(go.Scatter(
    x=datos.index,
    y=datos['EMA_9'],
    line=dict(color='green', width=1.5),
    name='EMA 9'
))

# EMA 21
fig.add_trace(go.Scatter(
    x=datos.index,
    y=datos['EMA_21'],
    line=dict(color='blue', width=1.5),
    name='EMA 21'
))

# Bandas de Bollinger
fig.add_trace(go.Scatter(
    x=datos.index,
    y=datos['BB_upper'],
    line=dict(color='lightgray', width=1),
    name='Bollinger Superior',
    opacity=0.5
))
fig.add_trace(go.Scatter(
    x=datos.index,
    y=datos['BB_middle'],
    line=dict(color='gray', width=1),
    name='Bollinger Media',
    opacity=0.5
))
fig.add_trace(go.Scatter(
    x=datos.index,
    y=datos['BB_lower'],
    line=dict(color='lightgray', width=1),
    name='Bollinger Inferior',
    opacity=0.5
))

# === Layout principal ===
fig.update_layout(
    title='Gráfico de Velas + Indicadores Técnicos',
    width=1100,
    height=700,
    template='plotly_white',
    xaxis_title='Fecha',
    yaxis_title='Precio',
    xaxis_rangeslider_visible=False,  # Sacamos el rango inferior
    margin=dict(l=40, r=40, t=80, b=40),
    font=dict(family='Calibri', size=14),
)

# === Agregar RSI como un subplot ===
from plotly.subplots import make_subplots

# Redefinimos con subplots: 2 filas
fig_final = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    row_heights=[0.7, 0.3],
    specs=[[{"type": "xy"}], [{"type": "xy"}]]
)

# Velas y medias en la fila 1
for trace in fig.data:
    fig_final.add_trace(trace, row=1, col=1)

# RSI en la fila 2
fig_final.add_trace(go.Scatter(
    x=datos.index,
    y=datos['RSI'],
    line=dict(color='purple', width=2),
    name='RSI'
), row=2, col=1)

# Línea 70 y 30 en RSI
fig_final.add_hline(y=70, line_dash='dash', line_color='red', row=2, col=1)
fig_final.add_hline(y=30, line_dash='dash', line_color='green', row=2, col=1)

# === Layout final ===
fig_final.update_layout(
    title='Gráfico de Velas + Bandas de Bollinger + RSI',
    width=1100,
    height=800,
    template='plotly_white',
    xaxis_title='Fecha',
    yaxis_title='Precio',
    xaxis2_title='Fecha',
    yaxis2_title='RSI',
    xaxis_rangeslider_visible=False,
    margin=dict(l=40, r=40, t=80, b=40),
    font=dict(family='Calibri', size=14),
)

# === Mostrar
fig_final.show()

2

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import ta
from plotly.subplots import make_subplots

# === Tu DataFrame ===
# datos = pd.read_csv('archivo.csv', index_col='Date', parse_dates=True)

datos.index = pd.to_datetime(datos.index)
datos = datos[datos.index.dayofweek < 5]  # Solo días hábiles

# === Indicadores técnicos ===
# EMA
datos['EMA9'] = datos['Close'].ewm(span=9, adjust=False).mean()
datos['EMA21'] = datos['Close'].ewm(span=21, adjust=False).mean()

# Bollinger Bands
bb = ta.volatility.BollingerBands(close=datos['Close'], window=20, window_dev=2)
datos['BB_upper'] = bb.bollinger_hband()
datos['BB_middle'] = bb.bollinger_mavg()
datos['BB_lower'] = bb.bollinger_lband()

# RSI
datos['RSI'] = ta.momentum.RSIIndicator(close=datos['Close'], window=14).rsi()

# === Señales de Compra/Venta ===
datos['signal'] = 0
datos['signal'][9:] = \
    (datos['EMA9'][9:] > datos['EMA21'][9:]) & (datos['EMA9'][9:].shift(1) <= datos['EMA21'][9:].shift(1))

datos['exit'] = 0
datos['exit'][9:] = \
    (datos['EMA9'][9:] < datos['EMA21'][9:]) & (datos['EMA9'][9:].shift(1) >= datos['EMA21'][9:].shift(1))

# === Gráfico base con subplots ===
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    row_heights=[0.6, 0.2, 0.2],
    specs=[[{"type": "xy"}], [{"type": "xy"}], [{"type": "xy"}]]
)

# === Gráfico de Velas ===
fig.add_trace(go.Candlestick(
    x=datos.index,
    open=datos['Open'],
    high=datos['High'],
    low=datos['Low'],
    close=datos['Close'],
    name='Velas'
), row=1, col=1)

# EMA 9
fig.add_trace(go.Scatter(
    x=datos.index,
    y=datos['EMA9'],
    line=dict(color='green', width=1.5),
    name='EMA 9'
), row=1, col=1)

# EMA 21
fig.add_trace(go.Scatter(
    x=datos.index,
    y=datos['EMA21'],
    line=dict(color='blue', width=1.5),
    name='EMA 21'
), row=1, col=1)

# Bandas de Bollinger
fig.add_trace(go.Scatter(
    x=datos.index,
    y=datos['BB_upper'],
    line=dict(color='lightgray', width=1),
    name='BB Upper',
    opacity=0.4
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=datos.index,
    y=datos['BB_middle'],
    line=dict(color='gray', width=1),
    name='BB Middle',
    opacity=0.4
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=datos.index,
    y=datos['BB_lower'],
    line=dict(color='lightgray', width=1),
    name='BB Lower',
    opacity=0.4
), row=1, col=1)

# === Señales de compra ===
fig.add_trace(go.Scatter(
    x=datos.index[datos['signal'] == 1],
    y=datos['Close'][datos['signal'] == 1],
    mode='markers',
    marker=dict(symbol='triangle-up', color='lime', size=12),
    name='Compra'
), row=1, col=1)

# === Señales de venta ===
fig.add_trace(go.Scatter(
    x=datos.index[datos['exit'] == 1],
    y=datos['Close'][datos['exit'] == 1],
    mode='markers',
    marker=dict(symbol='triangle-down', color='red', size=12),
    name='Venta'
), row=1, col=1)

# === Volumen ===
fig.add_trace(go.Bar(
    x=datos.index,
    y=datos['Close'].diff().fillna(0).apply(lambda x: 1 if x > 0 else -1) * datos['Open'],
    marker_color=datos['Close'].diff().apply(lambda x: 'green' if x > 0 else 'red'),
    name='Volumen'
), row=2, col=1)

# === RSI ===
fig.add_trace(go.Scatter(
    x=datos.index,
    y=datos['RSI'],
    line=dict(color='purple', width=2),
    name='RSI'
), row=3, col=1)

# Líneas horizontales en RSI
fig.add_hline(y=70, line_dash='dash', line_color='red', row=3, col=1)
fig.add_hline(y=30, line_dash='dash', line_color='green', row=3, col=1)

# === Soportes y Resistencias Manuales (opcional) ===
fig.add_hline(y=1500, line_dash='dot', line_color='blue', row=1, col=1, annotation_text="Resistencia 1500", annotation_position="top left")
fig.add_hline(y=1000, line_dash='dot', line_color='blue', row=1, col=1, annotation_text="Soporte 1000", annotation_position="bottom left")

# === Layout Final ===
fig.update_layout(
    title='🔵 Gráfico Profesional de Velas + EMAs + RSI + Volumen + Señales',
    width=1200,
    height=1000,
    template='plotly_white',
    xaxis_rangeslider_visible=False,
    margin=dict(l=40, r=40, t=60, b=40),
    font=dict(family='Calibri', size=14),
    showlegend=True,
)

# Mostrar
fig.show()

3

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp


# 1. Filtrar solo días hábiles
datos = datos[datos.index.dayofweek < 5]

# 2. Crear figura con subplots: velas, volumen, y RSI
fig = sp.make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    row_heights=[0.6, 0.2, 0.2],
    subplot_titles=("Precio", "Volumen", "RSI")
)

# 3. Gráfico de velas
fig.add_trace(go.Candlestick(
    x=datos.index.astype(str),  # Convertimos a str para 'category'
    open=datos['Open'],
    high=datos['High'],
    low=datos['Low'],
    close=datos['Close'],
    name='Velas',
    increasing_line_color='blue',
    decreasing_line_color='red',
), row=1, col=1)

# 4. EMAs
for period, color in zip([5, 20, 50], ['green', 'orange', 'purple']):
    datos[f'EMA_{period}'] = datos['Close'].ewm(span=period).mean()
    fig.add_trace(go.Scatter(
        x=datos.index.astype(str),
        y=datos[f'EMA_{period}'],
        mode='lines',
        line=dict(color=color, width=2),
        name=f'EMA {period}'
    ), row=1, col=1)

# 5. Volumen
fig.add_trace(go.Bar(
    x=datos.index.astype(str),
    y=datos['Volume'],
    name='Volumen',
    marker_color='lightblue'
), row=2, col=1)

# 6. RSI
delta = datos['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
rsi = 100 - (100 / (1 + rs))
datos['RSI'] = rsi

fig.add_trace(go.Scatter(
    x=datos.index.astype(str),
    y=datos['RSI'],
    mode='lines',
    line=dict(color='black', width=2),
    name='RSI'
), row=3, col=1)

# 7. Líneas horizontales RSI
fig.add_hline(y=70, line_dash='dash', line_color='red', row=3, col=1)
fig.add_hline(y=30, line_dash='dash', line_color='green', row=3, col=1)

# 8. Layout final super pro
fig.update_layout(
    title='🔵 Gráfico Profesional de Velas + EMAs + RSI + Volumen + Sin días no hábiles',
    width=1200,
    height=1000,
    template='plotly_white',
    xaxis_rangeslider_visible=False,
    margin=dict(l=40, r=40, t=60, b=40),
    font=dict(family='Calibri', size=14),
    showlegend=True,
    xaxis=dict(
        type='category',  # <-- IMPORTANTE PARA NO TENER ESPACIOS VACIOS
        title='Fecha'
    )
)

# 9. Mostrar
fig.show()

4

In [ ]:
# EMAs
datos['EMA_4'] = datos['Close'].ewm(span=4).mean()
datos['EMA_9'] = datos['Close'].ewm(span=9).mean()
datos['EMA_40'] = datos['Close'].ewm(span=40).mean()

# Bandas de Bollinger
bb_period = 20
bb_std = 2
rolling_mean = datos['Close'].rolling(window=bb_period).mean()
rolling_std = datos['Close'].rolling(window=bb_period).std()
datos['BB_upper'] = rolling_mean + (rolling_std * bb_std)
datos['BB_lower'] = rolling_mean - (rolling_std * bb_std)

# --- Crear figura con 2 filas: precios arriba, volumen abajo ---
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    row_heights=[0.7, 0.3],
    vertical_spacing=0.05,
    subplot_titles=["Precio y Técnicos", "Volumen"]
)

# --- Gráfico de velas ---
fig.add_trace(
    go.Candlestick(
        x=datos.index,
        open=datos['Open'],
        high=datos['High'],
        low=datos['Low'],
        close=datos['Close'],
        name='Precio',
        increasing_line_color='blue',
        decreasing_line_color='red',
        increasing_fillcolor='blue',
        decreasing_fillcolor='red',
        increasing_line_width=1,
        decreasing_line_width=1
    ),
    row=1, col=1
)

# --- EMAs ---
ema_colors = ['green', 'lightgreen', 'yellow']
ema_names = ['EMA 4', 'EMA 9', 'EMA 40']
for ema, color, name in zip(['EMA_4', 'EMA_9', 'EMA_40'], ema_colors, ema_names):
    fig.add_trace(
        go.Scatter(
            x=datos.index,
            y=datos[ema],
            mode='lines',
            name=name,
            line=dict(color=color, width=2, dash='solid')
        ),
        row=1, col=1
    )

# --- Bandas de Bollinger ---
fig.add_trace(
    go.Scatter(
        x=datos.index,
        y=datos['BB_upper'],
        line=dict(color='gray', width=1),
        name='BB Upper',
        hoverinfo='skip'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=datos.index,
        y=datos['BB_lower'],
        line=dict(color='gray', width=1),
        name='BB Lower',
        fill='tonexty',
        fillcolor='rgba(200,200,200,0.2)',
        hoverinfo='skip'
    ),
    row=1, col=1
)

# --- Líneas de soporte y resistencia ---
soporte = 95
resistencia = 150

for nivel, color, dash in zip([soporte, resistencia], ['green', 'red'], ['dash', 'dot']):
    fig.add_shape(
        type='line',
        x0=datos.index.min(), x1=datos.index.max(),
        y0=nivel, y1=nivel,
        xref='x', yref='y',
        line=dict(color=color, width=2, dash=dash),
        row=1, col=1
    )

# --- Volumen ---
fig.add_trace(
    go.Bar(
        x=datos.index,
        y=datos['Volume'],
        name='Volumen',
        marker_color='lightblue',
        opacity=0.5
    ),
    row=2, col=1
)

# --- Configurar Layout Final ---
fig.update_layout(
    title=dict(
        text="Análisis Técnico Profesional - Ejemplo",
        font=dict(size=24, family="Calibri", color="blue"),
        x=0.5
    ),
    font=dict(
        family="Calibri",
        size=14,
        color="black"
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    hovermode='x unified',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    margin=dict(l=40, r=40, t=80, b=40),
    xaxis=dict(
        rangeslider_visible=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        tickformat='%b %d',
    ),
    yaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        title='Precio'
    ),
    yaxis2=dict(
        title='Volumen',
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True
    )
)

# --- Mostrar gráfico ---
fig.show()

5

In [ ]:
datos.index = pd.to_datetime(datos.index)

# --- Cálculos de indicadores ---
datos['EMA_4'] = datos['Close'].ewm(span=4).mean()
datos['EMA_9'] = datos['Close'].ewm(span=9).mean()
datos['EMA_40'] = datos['Close'].ewm(span=40).mean()

# Bandas de Bollinger
bb_period = 20
bb_std = 2
rolling_mean = datos['Close'].rolling(window=bb_period).mean()
rolling_std = datos['Close'].rolling(window=bb_period).std()
datos['BB_upper'] = rolling_mean + (rolling_std * bb_std)
datos['BB_lower'] = rolling_mean - (rolling_std * bb_std)

# --- Crear figura ---
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.7, 0.3],
    subplot_titles=["Precio y Técnicos", "Volumen"]
)

# --- Candlestick ---
fig.add_trace(
    go.Candlestick(
        x=datos.index,
        open=datos['Open'],
        high=datos['High'],
        low=datos['Low'],
        close=datos['Close'],
        name='Precio',
        increasing_line_color='lime',
        decreasing_line_color='red',
        increasing_fillcolor='lime',
        decreasing_fillcolor='red',
        increasing_line_width=1,
        decreasing_line_width=1
    ),
    row=1, col=1
)

# --- EMAs ---
ema_colors = ['cyan', 'orange', 'yellow']
ema_names = ['EMA 4', 'EMA 9', 'EMA 40']
for ema, color, name in zip(['EMA_4', 'EMA_9', 'EMA_40'], ema_colors, ema_names):
    fig.add_trace(
        go.Scatter(
            x=datos.index,
            y=datos[ema],
            mode='lines',
            name=name,
            line=dict(color=color, width=2, dash='solid')
        ),
        row=1, col=1
    )

# --- Bandas de Bollinger ---
fig.add_trace(
    go.Scatter(
        x=datos.index,
        y=datos['BB_upper'],
        line=dict(color='gray', width=1),
        name='BB Upper',
        hoverinfo='skip'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=datos.index,
        y=datos['BB_lower'],
        line=dict(color='gray', width=1),
        name='BB Lower',
        fill='tonexty',
        fillcolor='rgba(150,150,150,0.2)',
        hoverinfo='skip'
    ),
    row=1, col=1
)

# --- Líneas de soporte y resistencia ---
soporte = 95
resistencia = 150

for nivel, color, dash in zip([soporte, resistencia], ['green', 'red'], ['dash', 'dot']):
    fig.add_shape(
        type='line',
        x0=datos.index.min(), x1=datos.index.max(),
        y0=nivel, y1=nivel,
        xref='x', yref='y',
        line=dict(color=color, width=2, dash=dash)
    )

# --- Volumen ---
fig.add_trace(
    go.Bar(
        x=datos.index,
        y=datos['Volume'],
        name='Volumen',
        marker_color='lightblue',
        opacity=0.5
    ),
    row=2, col=1
)

# --- Anotaciones ---
fig.add_annotation(
    x=datos.index[-20],
    y=datos['Close'].iloc[-20],
    text="Zona de Compra",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40,
    bgcolor="gray",
    opacity=0.8,
    font=dict(color="white")
)

fig.add_annotation(
    x=datos.index[-5],
    y=datos['Close'].iloc[-5],
    text="Breakout",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40,
    bgcolor="gray",
    opacity=0.8,
    font=dict(color="white")
)

# --- Layout dark ---
fig.update_layout(
    width=1200, height=900, margin=dict(l=40, r=40, t=80, b=40),
    title=dict(
        text="Análisis Técnico Profesional - Modo Oscuro",
        font=dict(size=24, family="Calibri", color="white"),
        x=0.5
    ),
    font=dict(
        family="Calibri",
        size=14,
        color="white"
    ),
    plot_bgcolor='black',
    paper_bgcolor='black',
    hovermode='x unified',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1,
        font=dict(color="white")
    ),
    xaxis=dict(
        type='category',  # Solo días con datos
        showline=True,
        linewidth=1,
        linecolor='white',
        mirror=True
    ),
    yaxis=dict(
        title='Precio',
        showline=True,
        linewidth=1,
        linecolor='white',
        mirror=True
    ),
    yaxis2=dict(
        title='Volumen',
        showline=True,
        linewidth=1,
        linecolor='white',
        mirror=True
    )
)

# --- Mostrar ---
fig.show()

# --- Exportar a HTML ---
fig.write_html("grafico_tecnico_dark.html")

Grafico con lo util de los grafs 3, 4 y 5

In [ ]:
datos = data.GGAL[['Open', 'High', 'Low', 'Close', 'Volume']]
datos.head()

,Open,High,Low,Close,Volume
Date,,,,,
2023-12-11,1685.0,1783.750000,1660.0,1756.349976,4215047
2023-12-12,1787.0,1833.750000,1733.5,1780.849976,1965001
2023-12-13,1830.0,1921.000000,1705.0,1827.300049,3411926
2023-12-14,1835.0,1903.000000,1780.0,1788.849976,3493588
2023-12-15,1790.0,1799.949951,1640.0,1652.500000,3948715


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ta
from src.quant_fi import calcular_ema


# --- Cálculos de indicadores ---
# EMA
# datos['EMA_4'] = datos['Close'].ewm(span=4).mean()
# datos['EMA_9'] = datos['Close'].ewm(span=9).mean()
# datos['EMA_40'] = datos['Close'].ewm(span=40).mean()

# datos['EMA_9'] = datos['Close'].ewm(span=9, adjust=False).mean()
# datos['EMA_21'] = datos['Close'].ewm(span=21, adjust=False).mean()

ema_periods = [4, 9, 40]
datos = calcular_ema(datos, columna='Close', window=4)
datos = calcular_ema(datos, columna='Close', window=9)
datos = calcular_ema(datos, columna='Close', window=40, shift_n=-4)


# Bandas de Bollinger
bb_period = 20
bb_std = 2
rolling_mean = datos['Close'].rolling(window=bb_period).mean()
rolling_std = datos['Close'].rolling(window=bb_period).std()
datos['BB_upper'] = rolling_mean + (rolling_std * bb_std)
datos['BB_lower'] = rolling_mean - (rolling_std * bb_std)

# bb = ta.volatility.BollingerBands(close=datos['Close'], window=20, window_dev=2)
# datos['BB_upper'] = bb.bollinger_hband()
# datos['BB_middle'] = bb.bollinger_mavg()
# datos['BB_lower'] = bb.bollinger_lband()

# RSI
# datos['RSI'] = ta.momentum.RSIIndicator(close=datos['Close'], window=14).rsi()

# --- Líneas de soporte y resistencia ---
soporte = 5000
resistencia = 8700

datos

,Open,High,Low,Close,Volume,EMA_4,EMA_9,EMA_40,BB_upper,BB_lower
Date,,,,,,,,,,
2023-12-11,1685.0,1783.750000,1660.0,1756.349976,4215047,NaN,NaN,NaN,NaN,NaN
2023-12-12,1787.0,1833.750000,1733.5,1780.849976,1965001,NaN,NaN,NaN,NaN,NaN
2023-12-13,1830.0,1921.000000,1705.0,1827.300049,3411926,NaN,NaN,NaN,NaN,NaN
2023-12-14,1835.0,1903.000000,1780.0,1788.849976,3493588,1788.337494,NaN,NaN,NaN,NaN
2023-12-15,1790.0,1799.949951,1640.0,1652.500000,3948715,1734.002496,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-04-25,7520.0,7730.000000,7520.0,7610.000000,2022374,7498.631475,7339.383646,7211.353586,8016.620246,6221.379754
2025-04-28,7610.0,7690.000000,7310.0,7430.000000,1468569,7471.178885,7357.506917,NaN,8010.156592,6222.843408
2025-04-29,7420.0,7470.000000,7240.0,7290.000000,1631781,7398.707331,7344.005534,NaN,8008.289243,6222.710757


In [ ]:


assets_graph = len(datos.columns)
rows_graph = (assets_graph // 2) + (assets_graph % 2)
cols_graph = 2

In [ ]:
datos = datos
fecha_inicio, fecha_fin = datos.index[[0, -1]]

periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"
graf_titulo = f'{tickers[0]}  |  Período {periodo}'
eje_x_titulo = f'Fecha'
eje_y_titulo = f'Precio'
eje_x_titulo_2 = f'Fecha'
eje_y_titulo_2 = f'Volumen'

eje_x = datos.index
primeros_dias = datos.groupby([datos.index.year, datos.index.month]).first().index
# Es un MultiIndex (year, month), lo convertimos en fechas
primeros_dias = [datos.loc[(datos.index.year == year) & (datos.index.month == month)].index.min() for year, month in primeros_dias]
primer_dia = eje_x.min()
ultimo_dia = eje_x.max()
fechas_tick = pd.to_datetime(primeros_dias + [primer_dia, ultimo_dia])
fechas_tick = fechas_tick.drop_duplicates().sort_values()

tickvals = fechas_tick.strftime('%Y/%m/%d')
ticktext = fechas_tick.strftime('%Y-%m-%d')

colores_volumen = np.where(datos['Close'] >= datos['Open'], get_color(2), get_color(3))

# fig = go.Figure()
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.095, row_heights=[0.80, 0.20], specs=[[{"secondary_y": True}], [{"secondary_y": False}]]
)

fig.add_trace(
    go.Candlestick(
        x=eje_x.strftime('%Y/%m/%d'), open=datos['Open'], high=datos['High'], low=datos['Low'], close=datos['Close'], name=tickers[0],
        increasing_line_color=get_color(2), decreasing_line_color=get_color(3),
        increasing_line_width=1, decreasing_line_width=1
    ), row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=eje_x.strftime('%Y/%m/%d'), y=datos['Close'], mode='lines', line=dict(width=0),showlegend=False, hoverinfo='skip'
    ), row=1, col=1, secondary_y=True
)

fig.add_trace(
    go.Bar(
        x=eje_x.strftime('%Y/%m/%d'), y=datos['Volume'], name='Volumen', showlegend=False, marker_color=colores_volumen, opacity=0.7, 
    ), row=2, col=1
)
# Utimo precio
fig.add_shape(
	type="line", x0=min(eje_x.strftime('%Y/%m/%d')), x1=max(eje_x.strftime('%Y/%m/%d')), 
	y0=datos['Close'].iloc[-1], y1=datos['Close'].iloc[-1],
	line=dict(color=get_color(4), width=1, dash="dot"), row=1, col=1)

# # === Volumen ===
# fig.add_trace(go.Bar(
#     x=datos.index,
#     y=datos['Close'].diff().fillna(0).apply(lambda x: 1 if x > 0 else -1) * datos['Open'],
#     marker_color=datos['Close'].diff().apply(lambda x: 'green' if x > 0 else 'red'),
#     name='Volumen'
# ), row=2, col=1)

# --- EMAs ---
# ema_colors = ['green', 'lightgreen', 'yellow']
# ema_colors = ['cyan', 'orange', 'yellow']
# ema_colors = ['green', 'orange', 'purple']
ema_colors = [get_color(12), get_color(10), get_color(1)]
for period, color in zip(ema_periods, ema_colors):
    fig.add_trace(
        go.Scatter(
            x=eje_x.strftime('%Y/%m/%d'),
            # y=datos[ema],
            y=datos[f'EMA_{period}'],
            #mode='lines',
            # name=name,
            name=f'EMA {period}',
            line=dict(color=color, width=2, dash='solid')
        ),
        row=1, col=1
    )
    
# 4. EMAs
# for period, color in zip([5, 20, 50], ['green', 'orange', 'purple']):
    # datos[f'EMA_{period}'] = datos['Close'].ewm(span=period).mean()
    # fig.add_trace(go.Scatter(
    #     x=datos.index.astype(str),
    #     y=datos[f'EMA_{period}'],
    #     mode='lines',
    #     line=dict(color=color, width=2),
    #     name=f'EMA {period}'
    # ), row=1, col=1)


# # --- Bandas de Bollinger ---
# fig.add_trace(
#     go.Scatter(
#         # x=datos.index,
#         x=eje_x,
#         y=datos['BB_upper'],
#         line=dict(color='gray', width=1),
#         name='BB Upper',
#         hoverinfo='skip', opacity=0.4
#     ),
#     row=1, col=1
# )

# fig.add_trace(
#     go.Scatter(
#         # x=datos.index,
#         x=eje_x,
#         y=datos['BB_lower'],
#         line=dict(color='gray', width=1),
#         name='BB Lower',
#         fill='tonexty',
#         fillcolor='rgba(150,150,150,0.2)',
#         hoverinfo='skip', opacity=0.4
#     ),
#     row=1, col=1
# )


# # === RSI ===
# fig.add_trace(go.Scatter(
#     x=datos.index,
#     y=datos['RSI'],
#     line=dict(color='purple', width=2),
#     name='RSI'
# ), row=3, col=1)

# # # Líneas horizontales en RSI
# fig.add_hline(y=70, line_dash='dash', line_color='red', row=3, col=1)
# fig.add_hline(y=30, line_dash='dash', line_color='green', row=3, col=1)

# # 6. RSI
# delta = datos['Close'].diff()
# gain = delta.clip(lower=0)
# loss = -delta.clip(upper=0)
# avg_gain = gain.rolling(window=14).mean()
# avg_loss = loss.rolling(window=14).mean()
# rs = avg_gain / avg_loss
# rsi = 100 - (100 / (1 + rs))
# datos['RSI'] = rsi

# fig.add_trace(go.Scatter(
#     x=datos.index.astype(str),
#     y=datos['RSI'],
#     mode='lines',
#     line=dict(color='black', width=2),
#     name='RSI'
# ), row=3, col=1)


# # === Soportes y Resistencias Manuales (opcional) ===
# fig.add_hline(y=1500, line_dash='dot', line_color='blue', row=1, col=1, annotation_text="Resistencia 1500", annotation_position="top left")
# fig.add_hline(y=1000, line_dash='dot', line_color='blue', row=1, col=1, annotation_text="Soporte 1000", annotation_position="bottom left")

# --- Líneas de soporte y resistencia ---
# for nivel, color, dash in zip([soporte, resistencia], ['green', 'red'], ['dash', 'dot']):
#     fig.add_shape(
#         type='line',
#         x0=datos.index.min(), x1=datos.index.max(),
#         # x=eje_x,
#         y0=nivel, y1=nivel,
#         xref='x', yref='y',
#         line=dict(color=color, width=2, dash=dash), row=1, col=1
#     )

# === Señales de compra ===
# fig.add_trace(go.Scatter(
#     x=datos.index[datos['signal'] == 1],
#     y=datos['Close'][datos['signal'] == 1],
#     mode='markers',
#     marker=dict(symbol='triangle-up', color='lime', size=12),
#     name='Compra'
# ), row=1, col=1)

# === Señales de venta ===
# fig.add_trace(go.Scatter(
#     x=datos.index[datos['exit'] == 1],
#     y=datos['Close'][datos['exit'] == 1],
#     mode='markers',
#     marker=dict(symbol='triangle-down', color='red', size=12),
#     name='Venta'
# ), row=1, col=1)

# --- Anotaciones ---
# fig.add_annotation(
#     x=datos.index[-20],
#     y=datos['Close'].iloc[-20],
#     text="Zona de Compra",
#     showarrow=True,
#     arrowhead=2,
#     ax=0,
#     ay=-40,
#     bgcolor="gray",
#     opacity=0.8,
#     font=dict(color="white")
# )

# fig.add_annotation(
#     x=datos.index[-5],
#     y=datos['Close'].iloc[-5],
#     text="Breakout",
#     showarrow=True,
#     arrowhead=2,
#     ax=0,
#     ay=-40,
#     bgcolor="gray",
#     opacity=0.8,
#     font=dict(color="white")
# )

fig.update_layout(
    width=1100, height=800, margin=dict(l=60, r=10, t=70, b=50),
    title=dict(text=graf_titulo, x=0.5, y=0.96),
    xaxis=dict(
        type='category', tickmode='array', tickvals=tickvals, ticktext=ticktext, tickangle=45, showticklabels=True, showline=True, linewidth=2,
	),
    yaxis=dict(
        title=dict(text=eje_y_titulo, standoff=10), type='linear', side='left', 
    ),
    yaxis2=dict(
		overlaying='y', side='right', showgrid=False, showticklabels=True, matches='y',
    ),
    xaxis2=dict(
		title=dict(text=eje_x_titulo_2, standoff=20),
        type='category', tickmode='array', tickvals=tickvals, showticklabels=False,
	),
    yaxis3=dict(
        title=dict(text=eje_y_titulo_2, standoff=10), side='left', showline=True, linewidth=1,
    ),
    # hovermode='x unified',
	legend=dict(borderwidth=1, x=0.01, y=0.99, xanchor="left", yanchor="top"),
    xaxis_rangeslider_visible=False,  # Sacamos el rango inferior
)


set_bloomberg_dark(fig)
# set_bloomberg_light(fig)
fig.show()
# fig.write_html("grafico_tecnico_dark.html")

NameError: name 'datos' is not defined

In [ ]:
# eje_x = datos.index
# dias_total = (fecha_fin - fecha_inicio).days
# dias_extra = int(dias_total * 0.01)
# x_min_extendido = fecha_inicio - pd.Timedelta(days=dias_extra)
# x_max_extendido = fecha_fin + pd.Timedelta(days=dias_extra)

# tickvals = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='MS').to_list()
# if fecha_inicio not in tickvals: tickvals.insert(0, fecha_inicio)
# if fecha_fin not in tickvals: tickvals.append(fecha_fin)


# max_ticks = 15
# tickvals = datos.index.strftime('%Y/%m/%d')
# n = max(1, len(tickvals) // max_ticks)
# tickvals = tickvals[::n]

# # --- Calcular tickvals automáticos para eje x ---

# # Calculamos cada cuántas fechas poner una etiqueta
# total_fechas = len(eje_x)
# salto_ticks = max(1, total_fechas // max_ticks)
# # Creamos los tickvals de forma automática
# # tickfechas = eje_x[::salto_ticks].tolist()
# tickfechas = [fecha.strftime('%Y-%m-%d') for fecha in eje_x[::salto_ticks]]
# # Nos aseguramos que la última fecha esté siempre incluida
# # if eje_x[-1] not in tickfechas:
#     # tickfechas.append(eje_x[-1])
# # Asegurarte de agregar también el último día como string

# if eje_x[-1].strftime('%Y-%m-%d') not in tickfechas:
#     tickfechas.append(eje_x[-1].strftime('%Y-%m-%d'))


In [ ]:
# # --- Asumimos que el índice es DatetimeIndex ---
# eje_x = datos.index

# # --- Detectamos rango de fechas ---
# rango_dias = (eje_x.max() - eje_x.min()).days
# rango_segundos = (eje_x.max() - eje_x.min()).total_seconds()

# # --- Definimos formato de fecha según el rango ---
# if rango_dias > 1095:  # más de 3 años
#     formato_tick = '%Y'
#     resample_rule = 'YS'  # Year Start
# elif rango_dias > 365:  # entre 1 y 3 años
#     formato_tick = '%b %Y'
#     resample_rule = 'MS'  # Month Start
# elif rango_dias > 31:  # entre 1 mes y 1 año
#     formato_tick = '%d %b'
#     resample_rule = 'W-MON'  # cada lunes (semana)
# elif rango_dias > 1:  # menos de 1 mes pero más de 1 día
#     formato_tick = '%d %b'
#     resample_rule = 'D'  # Día
# else:  # datos intradía
#     formato_tick = '%d %b %H:%M'
#     resample_rule = 'H'  # Hora

# # --- Definimos qué fechas serán los ticks ---
# fechas_tick = datos.resample(resample_rule).first().dropna().index

# # --- Agregar explícitamente primer y último día/hora ---
# fechas_tick = fechas_tick.union([eje_x.min(), eje_x.max()]).sort_values()

# # --- Preparamos tickvals y ticktext ---
# tickvals = fechas_tick.strftime('%d/%m/%Y %H:%M')
# ticktext = fechas_tick.strftime(formato_tick)

# # --- Graficamos ---
# fig = go.Figure()

# fig.add_trace(go.Candlestick(
#     x=eje_x.strftime('%d/%m/%Y %H:%M'),  # PARA PLOTLY: solo para el eje, NO tocamos el índice
#     open=datos['Open'],
#     high=datos['High'],
#     low=datos['Low'],
#     close=datos['Close'],
# ))

# fig.update_layout(
#     xaxis=dict(
#         type='category',       # ejes de strings
#         tickmode='array',
#         tickvals=tickvals,
#         ticktext=ticktext,
#         tickangle=45,
#     ),
#     width=1200,
#     height=600,
# )

# fig.show()

Graf basico

In [ ]:
fig = go.Figure()

fig.add_trace(go.Candlestick(
            x=datos.index,
            open=datos['Open'],
            high=datos['High'],
            low=datos['Low'],
            close=datos['Close']
        ))

# fig.add_trace(
#     go.Scatter(x=datos.index, y=datos['Close'], mode='lines', name='Precio')
# )

fig.show()

In [ ]:
import cufflinks as cf
cf.set_config_file(offline=True)
# lo usamos en la visualizacion custom
from IPython.display import display, HTML, Javascript

In [ ]:
# qf = cf.QuantFig(datos, kind='candle', title='Primer grafico Quant', name='NVDA')
# qf.add_sma([10, 40], width=2, color=['green', 'lightgreen'], legendgroup=True)
# # qf.add_volume()
# qf.add_macd()
# qf.iplot()

In [ ]:
# datos = pd.DataFrame({
#     'Open': [1685.0, 1787.0, 1830.0, 1835.0, 1790.0, 7860.0, 7520.0, 7610.0, 7420.0, 7170.0],
#     'High': [1783.75, 1833.75, 1921.0, 1903.0, 1799.949951, 7930.0, 7730.0, 7690.0, 7470.0, 7340.0],
#     'Low': [1660.0, 1733.5, 1705.0, 1780.0, 1640.0, 7540.0, 7520.0, 7310.0, 7240.0, 7070.0],
#     'Close': [1756.349976, 1780.849976, 1827.300049, 1788.849976, 1652.5, 7580.0, 7610.0, 7430.0, 7290.0, 7190.0],
# }, index=pd.to_datetime([
#     '2023-12-11', '2023-12-12', '2023-12-13', '2023-12-14', '2023-12-15',
#     '2025-04-24', '2025-04-25', '2025-04-28', '2025-04-29', '2025-04-30'
# ]))

# datos.columns = [col.lower() for col in datos.columns]

# # Crear el QuantFig
# qf = cf.QuantFig(
#     datos_2,
#     kind='candle',
#     # title="Gráfico de Velas del Activo",
#     # up_color="blue",
#     # down_color="red",
#     # legend='right'
# )



# # Crear la figura
# fig = qf.iplot(asFigure=True)

# fig = qf.figure()

# # Mostrar el gráfico
# fig.show()

# # Configurar detalles de layout
# fig.update_layout(
#     width=1000,
#     height=600,
#     margin=dict(l=50, r=50, t=50, b=50),
#     font=dict(family="calibri", size=14),
#     title=dict(text="Gráfico de Velas - Datos de ejemplo", x=0.5),
#     xaxis=dict(
#         title="Fecha",
#         type='category',
#         tickmode='array',
#         ticktext=datos.index.strftime('%Y-%m-%d'),
#         tickvals=list(range(len(datos.index)))  # porque es category
#     ),
#     yaxis=dict(
#         title="Precio",
#         range=[datos['Low'].min() - 100, datos['High'].max() + 100]
#     )
# )

# # Mostrar el gráfico
# fig.show()

In [ ]:
# # Crear el QuantFig
# qf5 = cf.QuantFig(
#     datos, 
# 	kind='candle',
#     title="Gráfico de Velas con Configuración Personalizada",
# 	name='NVDA',
#     up_color="blue", 
#     down_color="red",
#     legend="right"
# )

# # Indicadores
# qf5.add_ema([4, 9, 40], width=2, color=['green', 'lightgreen', 'yellow'], legendgroup=False)
# # qf5.add_volume()
# qf5.add_bollinger_bands(periods=20, boll_std=2, name='BB')

# # Generar la figura
# fig5 = qf5.iplot(asFigure=True)

# # # Líneas de soporte y resistencia
# soporte = 95
# resistencia = 150

# lines = [
#     {"y": soporte, "color": "green", "dash": "dash"},
#     {"y": resistencia, "color": "red", "dash": "dot"}
# ]

# # # Agregar las líneas de manera iterativa
# for line in lines:
#     fig5.add_shape(
#         type="line",
# 		# x0=0,
#         # x1=len(datos.index) - 1,
#         x0=datos.index.min(),
#     	x1=datos.index.max(),
#         y0=line["y"],
#         y1=line["y"],
#         xref='x',   # Muy importante para que sepa que es respecto al eje x
#     	yref='y',   # Igual para y
#         line=dict(color=line["color"], width=2, dash=line["dash"])
#     )

# # # Configuración de layout
# fig5.update_layout(
#     width=900,               # Dimensiones del gráfico
#     height=600,
#     margin=dict(l=50, r=50, t=50, b=50),  # Márgenes
#     showlegend=True,         # Mostrar leyenda
#     title=dict(              # Estilo del título
#         text="Gráfico de Velas - Ejemplo Optimizado y sin Dias No Laborables",
#         font=dict(size=24, family="calibri", color="blue") #, weight="bold")
#     ),
#     font=dict(               # Fuente global
#         family="calibri",
#         size=14,
#         color="black"
#     ),
# 	xaxis=dict(
# 		title='Fecha', 
# 		type='category',
# 		tickmode='array',
# 		ticktext=datos.index.strftime('%Y-%m-%d')
# 	),
#     yaxis=dict(              # Configuración del eje Y
# 		title='Precio',
#         range=[datos['Low'].min() - 10, datos['High'].max() + 10],      # Rango de precios
#         type="linear"         # Escala lineal
#     )
# )

# # Mostrar el gráfico
# fig5.show()